In [1]:
import sys
sys.path.append('../')
from setting import config_read

In [2]:
import pandas as pd
import owlready2
from map import ResourceMapper
from elasticsearch import Elasticsearch

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




In [3]:
config = config_read('../')

# Elasticsearch
server_ip = config['elasticsearch']['ip']
index_name = config['elasticsearch']['name']
es = Elasticsearch(server_ip)
rm = ResourceMapper(config)

In [4]:
user_input = '용감한 형제가 작곡한 노래의 장르는?'
resource_combinations = rm.process(user_input)

In [5]:
def get_type(resource):
    search_query = {"query":{"term":{"URI.keyword": resource}}}
    result = es.search(index=index_name, body=search_query)
    return result['hits']['hits'][0]['_source']['Type']

In [6]:
conceptual_graph = []

for rc in resource_combinations:
    conceptual_arc = []
    state_list = []
    rc = list(rc)

    for r in rc:
        r_type = get_type(r)
        if r_type in ['T_dp', 'T_op']:
            state_list.append('edge') 
        else:
            state_list.append('vertex') 

    # 마지막이 edge일 때
    if state_list[-1] == 'edge':
        state_list.append('vertex')
        rc.append('owl:Thing')

    # vertex - vertex 일때
    index_list = []
    for i in range(len(state_list)-1):
        if state_list[i] == 'vertex' and state_list[i+1] == 'vertex':
            index_list.append(i+1)
    
    for i in index_list:
        state_list.insert(i, 'edge')
        rc.insert(i, 'Any P')
    
    for offset in range(0,len(rc)-2, 2):
        conceptual_arc.append(tuple(rc[offset:offset+3]))
    
    conceptual_graph.append(conceptual_arc)

conceptual_graph

[[('skmo:용감한_형제', 'skmo:isComposedBy', 'mo:Track'),
  ('mo:Track', 'Any P', 'skmo:Genre')],
 [('skmo:용감한_형제', 'skmo:isComposedBy', 'mo:Track'),
  ('mo:Track', 'skmo:hasGenre', 'owl:Thing')]]